In [ ]:
!jupyter-nbconvert SQLAlchemy_for_chemists.ipynb --to slides --post serve

Introduction to SQLAlchemy and Jupyter Notebooks
==========================================

### John Hey

#### Adapted from material by:
- Lee Bergstrand
- Matt McInnes

**Note:** This tutorial is based off the [official SQLAlchemy ORM tutorial](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html). 

Introduction to SQLAlchemy for Chemists
==========================

Firstly we need to import sqlalchemy and begin creating our database:

In [ ]:
import sqlalchemy

# Connect to the database...
from sqlalchemy import create_engine

# We normally set echo=False, but here we want to see the sql we are produce later
engine = create_engine('sqlite:///:memory:', echo=True)

**Note** that the string ``` 'sqlite:///:memory:' ``` sets the backend to sqlite and creates a database in memory. (RAM) 

This is useful for testing but we can change it to save to the harddisk later if we want our data to be permenant.

To set this to a file, we can just replace `:memory:` with a file path i.e.:

```python
file_path = "/home/users/john/my_super_secret_database.db"
engine = create_engine('sqlite:///'+file_path, echo=True)```

### Creating Database-Backed Classes
---

Our next step is to define our classes which will be mapped onto tables within our database. 

We do this by defining a class in the normal pythonic way but with some extra SQLAlchemy commands. 

SQLAlchemy will then use this class to define and create the relavent tables.  

In SQLAlchemy the ORM utilizes classes inheriting from the SQLAlchemy base class. 

**Note:** ORMs (Object Relational Mapper) are classes which allow you to refer to data stored in your database from another language.

Columns within a database table can usually only store scalar data (Ints, Strings etc.) an ORM allows you to map more complex datatypes onto these columns within your database. 

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In our example we are going to create a database of molecules and structural motifs

Classes define RDBMS (Relational DataBase Management Scheme) table attributes:

- We must define the `__tablename__` attribute
- Then we define some `Column` attributes
>- Columns have datatypes (Integer, Float, etc.)
>- Columns have constraints (Primary Key, Foreign Key, etc.)

In [ ]:
from sqlalchemy import Column, Integer, String, PickleType

class Molecule(Base):
    __tablename__ = 'molecules'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    atoms = Column(PickleType)
    
    # Defines to_string() representation 
    def __repr__(self):
        return f"<Molecule(name={self.name}, atoms={self.atoms}, id={self.id})>"

SQLAlchemy will now set up your table for you...

In [ ]:
Base.metadata.create_all(engine)

##### Issued SQL:

```SQL
CREATE TABLE molecules (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	atoms BLOB, 
	PRIMARY KEY (id)
)
```

This SQL will look familiar to those of you that have interacted with SQL databases by hand

This instructs the database to create a new table with columns:
- `id`    -- an integer that cannot have no value (`NOT NULL`)
- `name`  -- a character array (i.e. a string) 
- `atoms` -- a binary blob, i.e. some arbitary data. 

It also sets the `id` column up as the primary key for this table.
- So this table will be indexed by the `id` column

### Creating a Session

---

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

The `session` object is created by the `sqlalchemy.orm.sessionmaker` class and bound to our database's engine. 

This `session` is our interface to the database. 
- We use this to add/remove/fetch items from the database. 

We will see this in action in the next section.

### Adding, Updating and Commiting Objects
---

Once you have your tables create you can then start creating objects.

In [ ]:
phenol = Molecule(name="phenol", atoms=["C", "C", "C", "C", "C", "C", "H", "H", "H", "H", "H", "O", "H"])
session.add(phenol)
print(phenol)

**Note:** No SQL has been printed here and `id=None` because we have not yet committed this object to the database

There are a couple of ways to commit your new object to the database:

- Call the `session.commit()` method, or;
- When you do a query SQLAlchemy, your uncommited changes are flushed to the db (within a transaction). SQLAlchemy then requeries the database in order to gain the value of the objects primary key column.

In [ ]:
our_molecule = session.query(Molecule).filter_by(name='phenol').first()

##### Issued SQL:

*Insert (Flush)*

```SQL
INSERT INTO molecules (name, atoms) VALUES ('phenol', <memory at 0x7effc62f4408>)
```

*Query Select*

```SQL
SELECT molecules.id AS molecules_id, 
        molecules.name AS molecules_name, 
        molecules.atoms AS molecules_atoms 
FROM molecules 
WHERE molecules.name = 'phenol'
 LIMIT 1 OFFSET 0```

Since both of `phenol` and `our_molecule` have the same primary key they are mapped to the same object.

In [ ]:
print(f"phenol = {phenol}")
print(f"our_molecule = {our_molecule}")

print(phenol is our_molecule)

Multiple objects can be added to the session and objects can be modified.

In [ ]:
session.add_all([
        Molecule(name="water", atoms=["O", "H", "H"]),
        Molecule(name="CO2", atoms=["O", "O", "C"]),
        Molecule(name="benzene", atoms=['C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H']),])

# We can modify the name if we decide that we want the systematic name instead
phenol.name = "Cyclohexa-1,3,5-trienol"  

Finally, the transaction can be commited. The remaining changes are flushed to the database.

In [ ]:
session.commit()

##### Issued SQL:

*Flush remaining changes (inserts and updates)*

```SQL
UPDATE molecules SET name='Cyclohexa-1,3,5-trienol' WHERE molecules.id = 1

INSERT INTO molecules (name, atoms) VALUES ('water', <memory at 0x7fefdf101408>)
INSERT INTO molecules (name, atoms) VALUES ('CO2', <memory at 0x7fefdf101588>)
INSERT INTO molecules (name, atoms) VALUES ('benzene', <memory at 0x7fefdf101408>)
```

*Finally, commit the changes*

```SQL
COMMIT
```

### Rolling Back
---

In SQLAlchemy all database operations are performed within a transaction. 

For example we can edit and query for objects:

In [ ]:
phenol.name = 'Phenol'

fake_molecule = Molecule(name='fakemol', atoms=["F", "Al", "Se"])
session.add(fake_molecule)

session.query(Molecule).filter(Molecule.name.in_(['Phenol', 'fakemol'])).all()

We can do a rollback and our changes are reset.

In [ ]:
session.rollback()

##### Issued SQL:

```SQL
ROLLBACK
```

Our objects are reset to their pervious states and our uncommited objects are removed.

In [ ]:
phenol.name

In [ ]:
fake_molecule in session

So our fake molecule is no longer part of our database session

### Querying
---

We can query our database for objects in the `Molecules` table and sort them

In [ ]:
for instance in session.query(Molecule).order_by(Molecule.id):
    print(instance)

##### Issued SQL:

```SQL
SELECT molecules.id AS molecules_id, 
        molecules.name AS molecules_name, 
        molecules.atoms AS molecules_atoms 
FROM molecules ORDER BY molecules.id
```

**Note:** This returns all the information associated with each Molecule object.

Sometimes we only want to get certain information from each row:

In [ ]:
for name, atoms in session.query(Molecule.name, Molecule.atoms):
    print(name, atoms)

##### Issued SQL:

```SQL
SELECT molecules.name AS molecules_name, 
        molecules.atoms AS molecules_atoms 
FROM molecules
```

### Common Filtering Operators

We can filter our query so that we only select some of the data

Here’s an overview of some common operators used by the `filter()` fucntion:
- equals
- not equals
- like
- in
- not in
- and
- or

#### equals:

In [ ]:
for molecule in session.query(Molecule).filter(Molecule.name == 'water'):
    print(molecule)

#### not equals:

In [ ]:
for molecule in session.query(Molecule).filter(Molecule.name != 'water'):
    print(molecule)

#### LIKE:

In [ ]:
# everything that matches wat*
for molecule in session.query(Molecule).filter(Molecule.name.like('wat%')):
    print(molecule)   

In [ ]:
# everything with an "o" in the name:
for molecule in session.query(Molecule).filter(Molecule.name.like('%o%')):
    print(molecule)

#### IN:

In [ ]:
for molecule in session.query(Molecule).filter(Molecule.name.in_(['Cyclohexa-1,3,5-trienol',
                                                                 'water'])):
    print(molecule)

#### NOT IN:

In [ ]:
# use "tilda~" for NOT IN
for molecule in session.query(Molecule).filter(~Molecule.name.in_(['Cyclohexa-1,3,5-trienol',
                                                                 'water'])):
    print(molecule)

#### AND:

In [ ]:
for molecule in session.query(Molecule).filter(Molecule.name.in_(['Cyclohexa-1,3,5-trienol',
                                                                 'water']),
                                              Molecule.atoms == ["O", "H", "H"]):
    print(molecule)

#### OR:

In [ ]:
from sqlalchemy import or_
for molecule in session.query(Molecule).filter(or_(Molecule.name == "water", Molecule.id >= 4)):
    print(molecule)

### Relationships
---

Multiple classes can be created and linked together using class relations.

In [ ]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address

Once primary and foreign keys are established, classes can be connected together via relationships.

In [ ]:
User.addresses = relationship("Address", order_by=Address.id, back_populates="user")

Base.metadata.create_all(engine) # Flush schema changes to the DBMS.

In the above mappings:
- ```user.addresses``` maps to a user's list of address objects.
- ```address.user``` maps to an address's parent user object.

We create a "Jack" object and it initially has no addresses.

In [ ]:
jack = User(name='jack', fullname='Jack Bean', password='gjffdd')
jack.addresses

We then add some address objects.

In [ ]:
jack.addresses = [Address(email_address='jack@google.com'), Address(email_address='j25@yahoo.com')]

In [ ]:
jack.addresses[1]

We see that the "Jack" object's first address's user parameter refers back to the "Jack" object.

In [ ]:
jack.addresses[1].user

When we add "Jack" to the session and commit, the two address objects of "Jack" are also committed in a cascading fashion.

In [ ]:
session.add(jack)

session.commit()

jack = session.query(User).filter_by(name='jack').one()
jack.addresses